In [28]:
import requests
import json
from lxml import html
from urllib.request import Request, urlopen
import unicodecsv as csv

## 1. Downloading the Site
Why not city?

In [23]:
def create_url(zipcode):
    # Creating Zillow URL based on the filter.
    url = "https://www.zillow.com/homes/{0}".format(zipcode)
    print(url)
    return url

In [33]:
# try this:
url = create_url(zipcode)
requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}) 

https://www.zillow.com/homes/M4V


<Response [200]>

In [35]:
# why passing a header?
x = requests.get(url) 
print(x.text)

<html><head><meta name="robots" content="noindex, nofollow"/><link href="https://www.zillowstatic.com/vstatic/80d5e73/static/css/z-pages/captcha.css" type="text/css" rel="stylesheet" media="screen"/><script>
        window._pxAppId = 'PXHYx10rg3';
        window._pxJsClientSrc = '/HYx10rg3/init.js';
        window._pxHostUrl = '/HYx10rg3/xhr';
        window._pxFirstPartyEnabled = true;
        window._pxreCaptchaTheme='light';
    </script><script type="text/javascript" src="https://captcha.px-cdn.net/PXHYx10rg3/captcha.js?a=c&amp;m=0"></script>
    <script>
        function getQueryString(name, url) {
            if (!url) url = window.location.href;
            name = name.replace(/[\[\]]/g, "\\$&");
            var regex = new RegExp("[?&]" + name + "(=([^&#]*)|&|#|$)"),
                results = regex.exec(url);
            if (!results) return null;
            if (!results[2]) return '';
            return decodeURIComponent(results[2].replace(/\+/g, " "));
        }
        doc

The urllib.request module defines functions and classes which help in opening URLs (mostly HTTP) in a complex world — basic and digest authentication, redirections, cookies and more.

In [36]:
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'}) 
webpage = urlopen(req).read()

## 2. Parsing the Content

In [38]:
parser = html.fromstring(webpage)
parser

<Element html at 0x7f8fd0052b80>

`parser` now contains the whole HTML file in a nice tree structure which we can go over two different ways: XPath and CSSSelect. XPath is a way of locating information in structured documents such as HTML or XML documents. 
- [example](http://econpy.pythonanywhere.com/ex/001.html)

In [42]:
# Pick the right section: 
search_results = parser.xpath("//div[@id='search-results']//article")
raw_json_data = parser.xpath('//script[@data-zrr-shared-data-key="mobileSearchPageStore"]//text()')

In [43]:
raw_json_data

['<!--{"queryState":{"mapBounds":{"west":-79.41441,"east":-79.391204,"south":43.674469,"north":43.696545},"regionSelection":[{"regionId":797568,"regionType":7}],"isMapVisible":true,"filterState":{"isAllHomes":{"value":true}}},"filterDefinitions":{"keywords":{"id":"keywords","shortId":"att","labels":{"default":"Keywords","tracking":"Keyword"},"sortOrder":2,"type":"String","defaultValue":{"value":""},"exposedPillEnabled":true},"isPublicSchool":{"id":"isPublicSchool","shortId":"schp","labels":{"default":"Public"},"type":"Boolean","defaultValue":{"value":true}},"isCityView":{"id":"isCityView","shortId":"cityv","labels":{"default":"City"},"type":"Boolean","defaultValue":{"value":false}},"isWaterfront":{"id":"isWaterfront","shortId":"wat","labels":{"default":"Waterfront"},"type":"Boolean","defaultValue":{"value":false}},"isCondo":{"id":"isCondo","shortId":"con","labels":{"default":"Condos/co-ops"},"type":"Boolean","defaultValue":{"value":true}},"isMountainView":{"id":"isMountainView","shortI

In [44]:
def clean(text):
    # clean up text string
    if text:
        return ' '.join(' '.join(text).split())
    return None

cleaned_data = clean(raw_json_data).replace('<!--', "").replace("-->", "")
cleaned_data

'{"queryState":{"mapBounds":{"west":-79.41441,"east":-79.391204,"south":43.674469,"north":43.696545},"regionSelection":[{"regionId":797568,"regionType":7}],"isMapVisible":true,"filterState":{"isAllHomes":{"value":true}}},"filterDefinitions":{"keywords":{"id":"keywords","shortId":"att","labels":{"default":"Keywords","tracking":"Keyword"},"sortOrder":2,"type":"String","defaultValue":{"value":""},"exposedPillEnabled":true},"isPublicSchool":{"id":"isPublicSchool","shortId":"schp","labels":{"default":"Public"},"type":"Boolean","defaultValue":{"value":true}},"isCityView":{"id":"isCityView","shortId":"cityv","labels":{"default":"City"},"type":"Boolean","defaultValue":{"value":false}},"isWaterfront":{"id":"isWaterfront","shortId":"wat","labels":{"default":"Waterfront"},"type":"Boolean","defaultValue":{"value":false}},"isCondo":{"id":"isCondo","shortId":"con","labels":{"default":"Condos/co-ops"},"type":"Boolean","defaultValue":{"value":true}},"isMountainView":{"id":"isMountainView","shortId":"m

In [45]:
json_data = json.loads(cleaned_data) # parse cleaned_data
json_data

{'queryState': {'mapBounds': {'west': -79.41441,
   'east': -79.391204,
   'south': 43.674469,
   'north': 43.696545},
  'regionSelection': [{'regionId': 797568, 'regionType': 7}],
  'isMapVisible': True,
  'filterState': {'isAllHomes': {'value': True}}},
 'filterDefinitions': {'keywords': {'id': 'keywords',
   'shortId': 'att',
   'labels': {'default': 'Keywords', 'tracking': 'Keyword'},
   'sortOrder': 2,
   'type': 'String',
   'defaultValue': {'value': ''},
   'exposedPillEnabled': True},
  'isPublicSchool': {'id': 'isPublicSchool',
   'shortId': 'schp',
   'labels': {'default': 'Public'},
   'type': 'Boolean',
   'defaultValue': {'value': True}},
  'isCityView': {'id': 'isCityView',
   'shortId': 'cityv',
   'labels': {'default': 'City'},
   'type': 'Boolean',
   'defaultValue': {'value': False}},
  'isWaterfront': {'id': 'isWaterfront',
   'shortId': 'wat',
   'labels': {'default': 'Waterfront'},
   'type': 'Boolean',
   'defaultValue': {'value': False}},
  'isCondo': {'id': 'isC

In [46]:
search_results = json_data.get('cat1').get('searchResults').get('listResults', [])
search_results

[{'zpid': '2076901916',
  'id': '2076901916',
  'imgSrc': 'https://photos.zillowstatic.com/fp/667ac296af033a49feb5e58b33893746-p_e.jpg',
  'hasImage': True,
  'detailUrl': 'https://www.zillow.com/homedetails/223-Saint-Clair-Ave-W-1107-Toronto-ON-M4V-0A5/2076901916_zpid/',
  'statusType': 'FOR_SALE',
  'statusText': 'Condo for sale',
  'countryCurrency': 'C$',
  'price': 'C$859,900',
  'unformattedPrice': 859900,
  'address': '223 Saint Clair Ave W #1107, Toronto, ON M4V 0A5',
  'addressStreet': '223 Saint Clair Ave W #1107',
  'addressCity': 'Toronto',
  'addressState': 'ON',
  'addressZipcode': 'M4V0A5',
  'isUndisclosedAddress': False,
  'beds': 2,
  'baths': 2.0,
  'area': None,
  'latLong': {'latitude': 43.685681, 'longitude': -79.404733},
  'isZillowOwned': False,
  'commute': None,
  'variableData': {'type': 'TIME_ON_INFO',
   'text': '2 hours ago',
   'data': {'isFresh': False}},
  'hdpData': {'homeInfo': {'zpid': 2076901916,
    'streetAddress': '223 Saint Clair Ave W #1107',
 

In [17]:
def get_data_from_json(raw_json_data):
    cleaned_data = clean(raw_json_data).replace('<!--', "").replace("-->", "")
    properties_list = []

    try:
        json_data = json.loads(cleaned_data) # parse cleaned_data
        search_results = json_data.get('cat1').get('searchResults').get('listResults', [])

        # find all the attributes from webpage source
        for properties in search_results:
            property_info = properties.get('hdpData', {}).get('homeInfo')
            address = property_info.get('streetAddress')
            city = property_info.get('city')
            state = property_info.get('state')
            postal_code = property_info.get('zipcode')
            lat = property_info.get('latitude')
            lon = property_info.get('longitude')
            days = property_info.get('daysOnZillow')
            price = properties.get('price')
            bedrooms = properties.get('beds')
            bathrooms = properties.get('baths')
            area = properties.get('area')
            info = f'{bedrooms} bds, {bathrooms} ba ,{area} sqft'
            broker = properties.get('brokerName')
            property_url = properties.get('detailUrl')
            title = properties.get('statusText')

            data = {'address': address,
                    'city': city,
                    'state': state,
                    'postal_code': postal_code,
                    'lat': lat,
                    'lon': lon,
                    'days': days,
                    'price': price,
                    'facts and features': info,
                    'real estate provider': broker,
                    'url': property_url,
                    'title': title}

            # list of all listings
            properties_list.append(data)
        return properties_list

    except ValueError:
        print("Invalid json")
        return None

In [19]:
def parse(zipcode):
    url = create_url(zipcode)
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'}) 
    webpage = urlopen(req).read()
    parser = html.fromstring(webpage)
    search_results = parser.xpath("//div[@id='search-results']//article")

    # search_results in json format  
    raw_json_data = parser.xpath('//script[@data-zrr-shared-data-key="mobileSearchPageStore"]//text()')
    properties_list = get_data_from_json(raw_json_data)
    return properties_list

In [21]:
def write_data_to_csv(data):
    with open("mls_data/all_properties.csv", 'wb') as csvfile:
        fieldnames = [
            'title', 
            'address', 
            'city', 
            'state', 
            'postal_code', 
            'lat',
            'lon',
            'price', 
            'days',
            'facts and features', 
            'real estate provider', 
            'url']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in data:
            writer.writerow(row)

In [24]:
zipcode_lst = ["M4V"]

# append zipcodes 
final_list = []
for zipcode in zipcode_lst:
    print ("Fetching data for %s" % (zipcode))
    scraped_data = parse(zipcode)
    for data in scraped_data:
        final_list.append(data)
    print ("Number of listing: ", len(scraped_data))  

# export data
write_data_to_csv(final_list)

Fetching data for M4V
https://www.zillow.com/homes/M4V
Number of listing:  19
